In [21]:
from pathlib import Path
from collections import Counter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re

plt.style.use("ggplot")


In [17]:
DATA_PATH = Path("../data/raw/legal_text_classification.csv")  # ajustá nombre
df = pd.read_csv(DATA_PATH)
df.head()


,case_id,case_outcome,case_title,case_text
0,Case1,cited,Alpine Hardwood (Aust) Pty Ltd v Hardys Pty Lt...,Ordinarily that discretion will be exercised s...
1,Case2,cited,Black v Lipovac [1998] FCA 699 ; (1998) 217 AL...,The general principles governing the exercise ...
2,Case3,cited,Colgate Palmolive Co v Cussons Pty Ltd (1993) ...,Ordinarily that discretion will be exercised s...
3,Case4,cited,Dais Studio Pty Ltd v Bullett Creative Pty Ltd...,The general principles governing the exercise ...
4,Case5,cited,Dr Martens Australia Pty Ltd v Figgins Holding...,The preceding general principles inform the ex...


In [18]:
df.info()
df.describe()
df['case_outcome'].value_counts()

<class 'pandas.DataFrame'>
RangeIndex: 24985 entries, 0 to 24984
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   case_id       24985 non-null  str  
 1   case_outcome  24985 non-null  str  
 2   case_title    24985 non-null  str  
 3   case_text     24809 non-null  str  
dtypes: str(4)
memory usage: 780.9 KB


case_outcome
cited            12219
referred to       4384
applied           2448
followed          2256
considered        1712
discussed         1024
distinguished      608
related            113
affirmed           113
approved           108
Name: count, dtype: int64

En este bloque se limpian los textos legales para que el modelo pueda analizarlos mejor. Convertimos todo a minúsculas para evitar que “Court” y “court” se consideren diferentes palabras. Eliminamos números y signos de puntuación porque no aportan valor semántico para la clasificación. Luego quitamos las “stopwords” (palabras muy comunes como “the”, “and”, “of”) porque aparecen en casi todos los textos y no ayudan a diferenciar categorías. Finalmente, creamos una nueva columna con el texto limpio que será la base del análisis.

In [19]:
import sys
print(sys.executable)


d:\LR-300\Documents\legal-text-classification\.venv\Scripts\python.exe


In [20]:
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()  # convertir a minúsculas
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # eliminar números y signos
    tokens = text.split()  # dividir por espacios
    tokens = [t for t in tokens if t not in stop_words]  # quitar stopwords
    return " ".join(tokens)

df['clean_text'] = df['case_text'].astype(str).apply(clean_text)
df[['case_text','clean_text']].head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LR-300\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


AttributeError: 'float' object has no attribute 'lower'

In [22]:
all_words = " ".join(df['clean_text']).split()
freq = Counter(all_words)

common = freq.most_common(20)

words, counts = zip(*common)

plt.figure(figsize=(10,5))
plt.bar(words, counts)
plt.xticks(rotation=70)
plt.title("Top 20 palabras más frecuentes")
plt.show()


KeyError: 'clean_text'

En este bloque convertimos el texto en números usando la técnica TF-IDF (Term Frequency – Inverse Document Frequency). Los modelos de machine learning no entienden texto directamente, así que necesitamos transformarlo en vectores numéricos. TF-IDF asigna mayor peso a palabras que son importantes en un documento pero no tan frecuentes en todos los documentos. Esto mejora la capacidad del modelo para distinguir categorías.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)

X = vectorizer.fit_transform(df['clean_text'])
y = df['case_outcome']